In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time

### Constants

In [12]:
#import location list
zip_df = pd.read_csv('./Shared repo/project-client_project/Data/random_1000_zips.csv')

In [13]:
# word remove list
rem_str_list = ['.',',','read more','Get 5% Cash Back when you dine here',
                            'Offers takeout and delivery','Start Order','Offers reservations',
                            'Find a Table','Start Order','Get Cash Back','Offers takeout','Located in']

clfts = ['restaurants','shopping','localservices'] ## ,'shopping','bar'] 
## 'nightlife','massage', 'localservices','hotelstravel', 'food','eventservices','beautysvc','shopping']

locations = list(zip_df.zipcode)[2:11] # Brooklyn #11206

prices = ['RestaurantsPriceRange2.1','RestaurantsPriceRange2.2',
               'RestaurantsPriceRange2.3','RestaurantsPriceRange2.4']

sort_option = 'review_count' # ['rating','distance']

deliverys = ['%2CPlatformDelivery&ytp_st=delivery_default', '%2CPlatformPickup']

base_url='https://www.yelp.com/search?'

### Generate URLs Function

In [14]:
def generate_urls(base_url,locations,clfts,prices,deliverys):
    url_list=[]
    for location in locations:
         for clft in clfts:
            for price in prices:
    #             for delivery in deliverys:
                    url = base_url + 'find_desc=' + clft + '&find_loc=' + str(location)
                    url = url +'&attrs=' + price 
    #                 url = url + delivery
                    url = url + '&sortby=' + sort_option
                    url_list.append(url)
    return url_list
#                 res = requests.get(url)
#                 soup = BeautifulSoup(res.content,'lxml')
                

### Request function

In [15]:
def get_webPages(input_url,start=0,finish=30,step=30,sleep=1,getAll=True):
    pages_content_list = []
    search_location = re.findall(r'find_loc=(.*)&attrs', input_url)[0]
    url = input_url + '&start=0'
    res = requests.get(url)
    soup = BeautifulSoup(res.content,'lxml')
    results = get_numb_results(soup)
    showing_beg = results[0]
    showing_end = results[1]
    showing_total = results[2]
    time.sleep(sleep/3)
    print('start',start,"finish",finish,"step",showing_end," of total:",showing_total)
    if getAll == False and showing_end > 0:
        for i in range(start,finish,showing_end):
            url = input_url + '&start=' + str(i)
            res = requests.get(url)
            soup = BeautifulSoup(res.content,'lxml')
            print(res.status_code,'i:',i,url)
            city,state = get_city_state(soup)  
            pages_content_list.extend(get_buisniss_info(soup,city,state,search_location))
            time.sleep(sleep)
            if showing_end == showing_total:  ## Breake after first page in case there is few total results
                break
    if getAll == True and showing_end > 0:  
        for i in range(0,showing_total,showing_end):
            url = ""
            url = input_url + '&start=' + str(i)
            res = requests.get(url)
#             print(res.status_code,url)
            soup = BeautifulSoup(res.content,'lxml')
            city,state = get_city_state(soup)
            pages_content_list.extend(get_buisniss_info(soup,city,state,search_location))
            time.sleep(sleep)
#             if showing_end == showing_total:
#                 print(i,showing_end,showing_total)                   
    return pages_content_list

### Parse page functions

In [16]:
def get_city_state(soup_):
    spans = soup_.find_all("span")
    for item in range(len(spans)):
        if 'queryLocation__' in spans[item].attrs['class'][0]:
            content = spans[item].contents[0].split(',')
            city = content[0]
            state = content[1].split(' ')[1]
#             print(content,city,state) 
    return city,state

def get_numb_results(soup_):
    ps = soup_.find_all("p")
    total = 0
    res=[0,0,1]
    for p in ps:
        if len(p.contents) > 0: ## if no results then len == 0
            text = p.contents[0]
            if 'Showing' in text:
                begi = int(re.findall(r' ([0-9])+-', text)[0])
                end = int(re.findall(r'-([0-9]+) ', text)[0])
                total = int(re.findall(r'of ([0-9]+)', text)[0])
                res = [begi,end,max(1,total)]
    return res

def get_buisniss_info(soup_,city,state,search_location):
    a_tags = soup_.find_all('a')
    info_list = []
#     print(len(a_tags))
    for i in range(0,len(a_tags)):
            elem = a_tags[i]         
            if 'href' in elem.attrs:     
                href = elem.attrs['href']
                if '/biz/' in href and '#popup:' not in href and '?hrid=' not in href:
                    if str(type(elem.contents[0])) == "<class 'bs4.element.NavigableString'>":
                        info=[]
                        div = a_tags[i].parent.parent.parent.parent.parent.parent.parent.parent
                        classes = div.find_all(class_=True)
                        classes_names = get_class_names(classes)
                        sec_atributes_class_name = [x for x in classes_names if 'secondaryAttributes__' in str(x)]
                        price_atributes_class_name = [x for x in classes_names if 'priceCategory__' in str(x)]
                        bus_name_atributes_class_name = [x for x in classes_names if 'businessName__' in str(x)] 
                        info.extend(get_secondary(bus_name_atributes_class_name,div))
                        info.extend(get_secondary(price_atributes_class_name,div))
                        info.extend(get_secondary(sec_atributes_class_name,div))
                        info.append(href)
                        info.extend([city,state,search_location])
                        info_list.append(info)                     
    return info_list

def get_secondary(class_name,div):
    divs_sec = div.find("div", class_= class_name)
    results = [x for x in divs_sec.stripped_strings]
    if 'priceCategory__' in class_name[0]:
        results = [results[0], [val for val in results[1:len(results)] if val not in rem_str_list]]
    elif 'businessName__' in class_name[0]:
        results = [results[0],[val for val in results[1:len(results)] if val not in rem_str_list][0]]
    elif 'secondaryAttributes__' in class_name[0]:
        phone = re.findall(r'([(][0-9]+[)].*?),', ','.join(results))
        if phone==[]:
            phone=''      
        if len(results)==2:
            results.append(results[1])
            results[1]=results[0]    
            results[0] = phone
        elif len(results)==1:
            results.append(results[0])
            results.append('')
            results[0] = phone   
    results = [item for item in results if item not in rem_str_list]
#     print(results)
    return results

def get_class_names(classes):
    classes_names = []
    for element in classes:
        classes_names.extend(element["class"])
    return classes_names

## Main

In [17]:
urls = generate_urls(base_url,locations,clfts,prices,deliverys)
all_pages = []
count = 0
for item in urls:
    page = get_webPages(item,start=0,finish=60,step=30,sleep=15,getAll=False)
    all_pages.extend(page)
    count += 1
    if count // 10 == 0:
        df = pd.DataFrame(all_pages)
        df.columns = ['col_'+str(i) for i in range(df.shape[1])]
        df.to_csv('./Shared repo/project-client_project/Data/newtest'+'.csv', index=False)
# final save
df = pd.DataFrame(all_pages)
df.columns = ['col_'+str(i) for i in range(df.shape[1])]
df.to_csv('./Shared repo/project-client_project/Data/newtest'+'.csv', index=False)

start 0 finish 60 step 30  of total: 510
200 i: 0 https://www.yelp.com/search?find_desc=restaurants&find_loc=35210&attrs=RestaurantsPriceRange2.1&sortby=review_count&start=0
200 i: 30 https://www.yelp.com/search?find_desc=restaurants&find_loc=35210&attrs=RestaurantsPriceRange2.1&sortby=review_count&start=30
start 0 finish 60 step 30  of total: 455
200 i: 0 https://www.yelp.com/search?find_desc=restaurants&find_loc=35210&attrs=RestaurantsPriceRange2.2&sortby=review_count&start=0
200 i: 30 https://www.yelp.com/search?find_desc=restaurants&find_loc=35210&attrs=RestaurantsPriceRange2.2&sortby=review_count&start=30
start 0 finish 60 step 29  of total: 29
200 i: 0 https://www.yelp.com/search?find_desc=restaurants&find_loc=35210&attrs=RestaurantsPriceRange2.3&sortby=review_count&start=0
start 0 finish 60 step 1  of total: 1
200 i: 0 https://www.yelp.com/search?find_desc=restaurants&find_loc=35210&attrs=RestaurantsPriceRange2.4&sortby=review_count&start=0
start 0 finish 60 step 10  of total: 1

200 i: 10 https://www.yelp.com/search?find_desc=shopping&find_loc=35674&attrs=RestaurantsPriceRange2.2&sortby=review_count&start=10
200 i: 20 https://www.yelp.com/search?find_desc=shopping&find_loc=35674&attrs=RestaurantsPriceRange2.2&sortby=review_count&start=20
200 i: 30 https://www.yelp.com/search?find_desc=shopping&find_loc=35674&attrs=RestaurantsPriceRange2.2&sortby=review_count&start=30
200 i: 40 https://www.yelp.com/search?find_desc=shopping&find_loc=35674&attrs=RestaurantsPriceRange2.2&sortby=review_count&start=40
200 i: 50 https://www.yelp.com/search?find_desc=shopping&find_loc=35674&attrs=RestaurantsPriceRange2.2&sortby=review_count&start=50
start 0 finish 60 step 8  of total: 8
200 i: 0 https://www.yelp.com/search?find_desc=shopping&find_loc=35674&attrs=RestaurantsPriceRange2.3&sortby=review_count&start=0
start 0 finish 60 step 2  of total: 2
200 i: 0 https://www.yelp.com/search?find_desc=shopping&find_loc=35674&attrs=RestaurantsPriceRange2.4&sortby=review_count&start=0
star

start 0 finish 60 step 1  of total: 1
200 i: 0 https://www.yelp.com/search?find_desc=shopping&find_loc=36088&attrs=RestaurantsPriceRange2.2&sortby=review_count&start=0
start 0 finish 60 step 0  of total: 1
start 0 finish 60 step 0  of total: 1
start 0 finish 60 step 1  of total: 1
200 i: 0 https://www.yelp.com/search?find_desc=localservices&find_loc=36088&attrs=RestaurantsPriceRange2.1&sortby=review_count&start=0
start 0 finish 60 step 1  of total: 1
200 i: 0 https://www.yelp.com/search?find_desc=localservices&find_loc=36088&attrs=RestaurantsPriceRange2.2&sortby=review_count&start=0
start 0 finish 60 step 0  of total: 1
start 0 finish 60 step 0  of total: 1
start 0 finish 60 step 30  of total: 195
200 i: 0 https://www.yelp.com/search?find_desc=restaurants&find_loc=36093&attrs=RestaurantsPriceRange2.1&sortby=review_count&start=0
200 i: 30 https://www.yelp.com/search?find_desc=restaurants&find_loc=36093&attrs=RestaurantsPriceRange2.1&sortby=review_count&start=30
start 0 finish 60 step 30

In [8]:
df = pd.DataFrame(all_pages)
df.columns = ['col_'+str(i) for i in range(df.shape[1])]
df.to_csv('./Shared repo/project-client_project/Data/newtest'+'.csv', index=False)

In [39]:
a = []
len(a) == 0

True

In [40]:
# for page in all_pages:
#     print(page)

In [212]:
df = pd.DataFrame(all_pages)
df.columns = ['col_'+str(i) for i in range(df.shape[1])]

In [49]:
df

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10
0,1,Char Burger,$,"[Hot Dogs, Burgers]",[(256) 505-4720],(256) 505-4720,3848 Al Highway 69,/biz/char-burger-guntersville?osq=restaurants,Arab,AL,35016
1,2,Rainbow 278 Cafe,$,"[American (Traditional), Breakfast & Brunch]",[(205) 429-2150],(205) 429-2150,78877 U S 278,/biz/rainbow-278-cafe-blountsville?osq=restaur...,Arab,AL,35016
2,3,Brandin Iron Steakhouse,$,"[Steakhouses, American (Traditional)]",[(256) 796-9118],(256) 796-9118,430 Welcome Rd,/biz/brandin-iron-steakhouse-fairview?osq=rest...,Arab,AL,35016
3,4,Waffle House,$,"[Diners, Breakfast & Brunch, American (Traditi...",[(256) 582-0730],(256) 582-0730,2053 Gunter Ave,/biz/waffle-house-guntersville-3?osq=restaurants,Arab,AL,35016
4,5,Midas Burger,$,[American (Traditional)],[(256) 586-3275],(256) 586-3275,221 N Brindlee Mountain Pkwy,/biz/midas-burger-arab?osq=restaurants,Arab,AL,35016
5,6,Pizza Hut,$,"[Pizza, Italian, Chicken Wings]",[(256) 582-6141],(256) 582-6141,14365 Hwy 431 S,/biz/pizza-hut-guntersville?osq=restaurants,Arab,AL,35016
6,7,Grumpy’s Pizza & Pasta Shop,$,"[Pizza, Pasta Shops]",[(256) 931-2335],(256) 931-2335,1054 N Brindlee Mtn Pkwy,/biz/grumpys-pizza-and-pasta-shop-arab?osq=res...,Arab,AL,35016
7,8,Wendy’s,$,"[Burgers, Fast Food]",[(256) 582-7972],(256) 582-7972,1701 Gunter Ave,/biz/wendys-guntersville?osq=restaurants,Arab,AL,35016
8,9,LeCroy’s,$,"[Barbeque, American (New)]",[(256) 498-7675],(256) 498-7675,848 Apple Grove Rd,/biz/lecroys-union-grove?osq=restaurants,Arab,AL,35016
9,10,McDonald’s,$,"[Fast Food, Burgers, Coffee & Tea]",[(256) 582-2805],(256) 582-2805,1745 Gunter Ave,/biz/mcdonalds-guntersville?osq=restaurants,Arab,AL,35016


In [46]:
df.shape

(88, 11)

In [214]:
display(df[df['col_10']!='10011'])
display(df[df['col_10']=='10011'])

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11
970,616,Hanok,$$,[Korean],Food Gallery 32,Koreatown,/biz/hanok-new-york?osq=restaurants,New York,NY,10011,None,None
971,616,Hanok,$$,[Korean],Food Gallery 32,Koreatown,/biz/food-gallery-32-new-york,New York,NY,10011,None,None
997,641,Sabi Sushi,$$,[Sushi Bars],The Pennsy,Midtown West,/biz/sabi-sushi-new-york-5?osq=restaurants,New York,NY,10011,None,None
998,641,Sabi Sushi,$$,[Sushi Bars],The Pennsy,Midtown West,/biz/the-pennsy-new-york,New York,NY,10011,None,None
1265,1,Strand Bookstore,$,"[Bookstores, Used Bookstore]","Union Square, Greenwich Village, East Village",828 Broadway,"New York, NY 10003",(212) 473-1452,/biz/strand-bookstore-new-york?osq=shopping,New York,NY,10011
1266,2,Jacks.,$,[Discount Store],Midtown West,110 W 32nd St,"New York, NY 10001",(212) 268-9962,/biz/jacks-new-york?osq=shopping,New York,NY,10011
1267,3,Forever 21,$,"[Accessories, Women's Clothing, Men's Clothing]","Union Square, Greenwich Village",40 E 14th St,"New York, NY 10003",(212) 228-0598,/biz/forever-21-new-york-2?osq=shopping,New York,NY,10011
1268,4,Flying Tiger Copenhagen,$,"[Home Decor, Arts & Crafts, Kitchen & Bath]",Flatiron,920 Broadway,"New York, NY 10010",(212) 777-1239,/biz/flying-tiger-copenhagen-new-york-2?osq=sh...,New York,NY,10011
1269,5,Crossroads Trading Co.,$,"[Women's Clothing, Men's Clothing, Used, Vinta...",Flatiron,24 W 26th St,"New York, NY 10010",(646) 398-7917,/biz/crossroads-trading-co-new-york?osq=shopping,New York,NY,10011
1270,6,Forever 21,$,"[Women's Clothing, Accessories, Men's Clothing]",Midtown West,50 W 34th St,"New York, NY 10001",(212) 564-2346,/biz/forever-21-new-york?osq=shopping,New York,NY,10011


,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11
0,1,Los Tacos No.1,$,[Tacos],(212) 256-0343,Chelsea Market,Meatpacking District,/biz/los-tacos-no-1-new-york?osq=restaurants,New York,NY,10011,None
1,1,Los Tacos No.1,$,[Tacos],(212) 256-0343,Chelsea Market,Meatpacking District,/biz/chelsea-market-new-york,New York,NY,10011,None
2,2,Woorijip Authentic Korean Food,$,[Korean],(212) 244-1115,12 W 32nd St,Koreatown,/biz/woorijip-authentic-korean-food-new-york?o...,New York,NY,10011,None
3,3,Bleecker Street Pizza,$,"[Pizza, Italian]",(212) 924-4466,69 7th Ave S,West Village,/biz/bleecker-street-pizza-new-york?osq=restau...,New York,NY,10011,None
4,4,BaoHaus,$,"[Chinese, Taiwanese]",(646) 669-8889,238 E 14th St,East Village,/biz/baohaus-new-york-2?osq=restaurants,New York,NY,10011,None
5,5,Wafels & Dinges,$,"[Belgian, Waffles, Food Trucks]",[(646) 257-2592],(646) 257-2592,Serving New York and the Surrounding Area,/biz/wafels-and-dinges-new-york?osq=restaurants,New York,NY,10011,None
6,6,Taim West Village,$,"[Vegetarian, Sandwiches, Falafel]",(212) 691-1287,222 Waverly Pl,West Village,/biz/taim-west-village-new-york?osq=restaurants,New York,NY,10011,None
7,7,Corner Bistro,$,"[Burgers, Sports Bars]",(212) 242-9502,331 W 4th St,West Village,/biz/corner-bistro-new-york?osq=restaurants,New York,NY,10011,None
8,8,Num Pang Kitchen,$,"[Cambodian, Sandwiches]",(646) 791-0439,28 E 12th St,Union Square,/biz/num-pang-kitchen-new-york-6?osq=restaurants,New York,NY,10011,None
9,9,Dos Toros Taqueria,$,[Mexican],(212) 677-7300,137 4th Ave,Union Square,/biz/dos-toros-taqueria-new-york?osq=restaurants,New York,NY,10011,None


In [184]:
# a_tags = soup.find_all('a')
# info_list = []
# print(len(a_tags))
# for i in range(0,len(a_tags)):
#         elem = a_tags[i]         
#         if 'href' in elem.attrs:     
#             href = elem.attrs['href']
#             if '/biz/' in href and '#popup:' not in href and '?hrid=' not in href:
#                 if str(type(elem.contents[0])) == "<class 'bs4.element.NavigableString'>":
#                     div = a_tags[i].parent.parent.parent.parent.parent.parent.parent.parent.find_all('div')[0]
#                     span = a_tags[i].parent.parent.parent.parent.parent.parent.parent.parent.find_all('span')[0]
#                     info = [x for x in div.stripped_strings]
#                     info.append(span.find('div')['aria-label'])
#                     info_list.append(info)
# #                     print(info)

In [185]:
#remove item if item is dot or comma, "cooment"
def clean_content_list2(information_list,rem_str_list):
    for ii in range(len(information_list)):
        for k,value in reversed(list(enumerate(information_list[ii]))):
            if value in rem_str_list:
                information_list[ii].pop(k)
    return information_list
        
def clean_content_list(information_list,rem_str_list):
    print(len(information_list))
    for i1 in range(len(information_list)):
            print(i1)
            
            for rem_str in rem_str_list:
                print(rem_str)
                if rem_str in info_list[i1]:
                    information_list[i1].remove(rem_str)
    return ''

In [187]:
# a_tags[14].parent.parent.parent.parent.parent.parent.parent.parent.find_all('span')[0].find('div')['aria-label']

In [189]:
result = get_webPages('Restaurants','Brooklyn',start=0,finish=9930,step=30,sleep=2)
result = clean_content_list2(result,rem_str_list)


200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=0
360
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=30
333
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=60
341
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=90
344
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=120
344
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=150
349
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=180
332
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=210
352
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=240
348
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=270
352
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Br

138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=2610
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=2640
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=2670
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=2700
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=2730
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=2760
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=2790
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=2820
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=2850
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=2880
138
200 https://www.yelp.com/search?find_desc=Rest

138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=5220
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=5250
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=5280
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=5310
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=5340
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=5370
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=5400
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=5430
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=5460
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=5490
138
200 https://www.yelp.com/search?find_desc=Rest

138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=7830
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=7860
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=7890
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=7920
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=7950
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=7980
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=8010
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=8040
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=8070
138
200 https://www.yelp.com/search?find_desc=Restaurants&find_loc=Brooklyn%2C%20NY&start=8100
138
200 https://www.yelp.com/search?find_desc=Rest

In [190]:
result_df = pd.DataFrame(result)

In [216]:
df.to_csv('./Data/test2.csv', index=False)

# Testing part!

In [19]:
## TESTING PART
res = requests.get('https://www.yelp.com/search?find_desc=nightlife&find_loc=10011&attrs=RestaurantsPriceRange2.1&sortby=review_count&start=0')
soup = BeautifulSoup(res.content,'lxml')
res.status_code

200

In [20]:
soup_ = soup
a_tags = soup_.find_all('a')
info_list = []
print(len(a_tags))
for i in range(0,len(a_tags)):
        elem = a_tags[i]         
        if 'href' in elem.attrs:     
            href = elem.attrs['href']
            if '/biz/' in href and '#popup:' not in href and '?hrid=' not in href:
                if str(type(elem.contents[0])) == "<class 'bs4.element.NavigableString'>":
                    info=[]
                    div = a_tags[i].parent.parent.parent.parent.parent.parent.parent.parent
                    classes = div.find_all(class_=True)
                    sec_atributes_class_name = [x for x in classes_names if 'secondaryAttributes__' in str(x)]
                    price_atributes_class_name = [x for x in classes_names if 'priceCategory__' in str(x)]
                    bus_name_atributes_class_name = [x for x in classes_names if 'businessName__' in str(x)] 
                    info.extend(get_secondary(bus_name_atributes_class_name,div))
                    info.extend(get_secondary(price_atributes_class_name,div))
                    info.extend(get_secondary(sec_atributes_class_name,div))
                    info.append(href)
                    info_list.append(info)    

309


NameError: name 'classes_names' is not defined

In [22]:
ps = soup_.find_all("p")
total = 0
for p in ps:
#     print(p.contents[0])
    if 'Showing' in p.contents[0]:
        text = p.contents[0]
        begi = re.findall(r' ([0-9])+-', text)[0]
        end = re.findall(r'-([0-9]+) ', text)[0]
        total = re.findall(r'of ([0-9]+)', text)[0]
        


In [23]:
print(begi,end,total)

1 30 42


In [95]:
text = 'Showing 1-30 of 42'
begi = re.findall(r' ([0-9])+-', text)[0]
end = re.findall(r'-([0-9]+) ', text)[0]
total = re.findall(r'of ([0-9]+)', text)[0]
print(begi,end,total)

1 30 42
